In [1]:
import numpy as np
import pysam
import pysamstats

In [2]:
fasta_filename = '/share/lanzarolab/users/abasu/reference/Anopheles-gambiae-PEST_CHROMOSOMES_AgamP4.fa'
window_size = 300
fasta = pysam.FastaFile(fasta_filename)

In [9]:
for chromosome in ("2R", "2L", "3R", "3L", "X"):
    num_windows = fasta.get_reference_length(chromosome) // window_size + 1
    accessibility_pass = np.zeros(fasta.get_reference_length(chromosome)+1, '?') # initialize bool array as all Fails. uses 1-based indexing.
    with open(f"accessibility_masks/accessibility.{chromosome}.mask", 'r') as mask:
        for line in mask:
            position = int(line.lstrip(chromosome + " "))
            accessibility_pass[position] = True # set array element to true if the base pair PASSes
    # set element to False if <90% of bases in the window PASS
    windowed_accessibility = [not np.mean(accessibility_pass[n: n+window_size]) < 0.9 for n in np.arange(1, num_windows*window_size, window_size)]
    with open(f"accessibility.{chromosome}_{window_size}.mask", 'wb') as window_mask:
        np.save(window_mask, np.array(windowed_accessibility))
    print(f"{100*np.count_nonzero(windowed_accessibility)/len(windowed_accessibility)}% of windows in {chromosome} were marked as accessible")

47.63710632655946% of windows in 2R were marked as accessible
37.72090818484576% of windows in 2L were marked as accessible
45.032593494834664% of windows in 3R were marked as accessible
44.21893207700941% of windows in 3L were marked as accessible
39.80765210119172% of windows in X were marked as accessible
